In [1]:
%%capture
!pip install torch torchvision matplotlib opencv-python opencv-python-headless pillow

In [ ]:
!nvidia-smi

In [4]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                rel_dir = os.path.relpath(root, folder_path)
                rel_file = os.path.join(rel_dir, file)
                zipf.write(os.path.join(root, file), arcname=rel_file)


folder_to_zip = '/workspace/weights' 
zip_output_path = '/workspace/weights.zip' 
zip_folder(folder_to_zip, zip_output_path)
print("Zipping complete!")

Zipping complete!


In [13]:
import zipfile
import os
def unzip_file(zip_path, extract_to):
    os.makedirs(extract_to, exist_ok=True)


    if not os.path.exists(zip_path):
        print(f"No file found at {zip_path}")
        return


    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
            print(f"Extracted all contents to {extract_to}")
    except zipfile.BadZipFile:
        print("Failed to open the file as a zip. It may be corrupted or improperly formatted.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


zip_path = '/workspace/frames.zip'  
extract_to = '/workspace/frames' 
unzip_file(zip_path, extract_to)

Extracted all contents to /workspace/frames


GAN for 2.1, using training1 folder which contains frames selected from 1.2

In [14]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.nn import DataParallel
from torchvision import datasets, transforms
import itertools
from PIL import Image
from torchvision.utils import save_image
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class ResidualBlock(nn.Module):
    """Residual Block with two convolutional layers and skip connections."""
    def __init__(self, input_features):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(input_features, input_features, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(input_features),
            nn.ReLU(inplace=True),
            nn.Conv2d(input_features, input_features, kernel_size=3, padding=1, bias=False),
            nn.InstanceNorm2d(input_features)
        )

    def forward(self, x):
        return x + self.conv_block(x)

class Generator(nn.Module):
    """generator with downsampling, residual blocks, and upsampling components."""
    def __init__(self, input_channels, output_channels, n_residual_blocks=9):
        super(Generator, self).__init__()

        # Initial convolutional block
        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_channels, 64, kernel_size=7),
                 nn.InstanceNorm2d(64),
                 nn.ReLU(inplace=True)]

        # Downsampling
        in_features = 64
        out_features = in_features * 2
        for _ in range(2):
            model += [nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1),
                      nn.InstanceNorm2d(out_features),
                      nn.ReLU(inplace=True)]
            in_features = out_features
            out_features = in_features * 2

        # Residual blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features // 2
        for _ in range(2):
            model += [nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1),
                      nn.InstanceNorm2d(out_features),
                      nn.ReLU(inplace=True)]
            in_features = out_features
            out_features = in_features // 2

        # Output layer
        model += [nn.ReflectionPad2d(3),
                  nn.Conv2d(64, output_channels, kernel_size=7),
                  nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    """discriminator with convolutional layers that classifies images as real or fake."""
    def __init__(self, input_channels):
        super(Discriminator, self).__init__()

        model = [nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1),
                 nn.LeakyReLU(0.2, inplace=True)]

        model += [nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
                  nn.InstanceNorm2d(128),
                  nn.LeakyReLU(0.2, inplace=True)]

        model += [nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
                  nn.InstanceNorm2d(256),
                  nn.LeakyReLU(0.2, inplace=True)]

        model += [nn.Conv2d(256, 512, kernel_size=4, padding=1),
                  nn.InstanceNorm2d(512),
                  nn.LeakyReLU(0.2, inplace=True)]

        # Flatten the output and apply sigmoid
        model += [nn.Conv2d(512, 1, kernel_size=4, padding=1), nn.Sigmoid()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        x = self.model(x)
        return torch.flatten(x, 1)  # Flatten for the loss calculation

# Initialization of networks
netG_A2B = DataParallel(Generator(input_channels=3, output_channels=3)).to(device)
netG_B2A = DataParallel(Generator(input_channels=3, output_channels=3)).to(device)
netD_A = DataParallel(Discriminator(input_channels=3)).to(device)
netD_B = DataParallel(Discriminator(input_channels=3)).to(device)

class GANLoss(nn.Module):
    """GAN loss: binary cross-entropy between real and fake samples."""
    def __init__(self, device='cpu'):
        super(GANLoss, self).__init__()
        self.device = device
        self.loss = nn.BCELoss()

    def __call__(self, prediction, target_is_real):
        target_value = 1.0 if target_is_real else 0.0
        target_tensor = torch.full_like(prediction, target_value, device=self.device)  # Ensure matching shape and device
        return self.loss(prediction, target_tensor)



criterion_GAN = GANLoss(device=device)
criterion_cycle = torch.nn.L1Loss().to(device)
criterion_identity = torch.nn.L1Loss().to(device)

# Setup optimizers
optimizer_G = optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = optim.Adam(netD_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = optim.Adam(netD_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

def is_image_file(filename):
    """Check if a file is an image and not in a checkpoints directory."""
    valid_extensions = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp']
    return any(filename.endswith(extension) for extension in valid_extensions) and '.ipynb_checkpoints' not in filename

# Define your transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

Using device: cuda


In [ ]:
# Load datasets
dataset_A = datasets.ImageFolder('/workspace/frames/frames/game', transform=transform)
dataset_B = datasets.ImageFolder('/workspace/frames/frames/movie', transform=transform)

total_batch_size = 48

loader_A = DataLoader(dataset_A, batch_size=total_batch_size, shuffle=True, num_workers=12)
loader_B = DataLoader(dataset_B, batch_size=total_batch_size, shuffle=True, num_workers=12)


# Ensure the output directory exists
output_directory = '/workspace/outputs2.1'
os.makedirs(output_directory, exist_ok=True)

losses_G, losses_D_A, losses_D_B = [], [], []

epochs = 100  # Number of epochs to train
for epoch in range(epochs):
    for i, (real_A, real_B) in enumerate(zip(loader_A, loader_B)):
        real_A = real_A[0].to(device)  # Move data to device
        real_B = real_B[0].to(device)

        # -----------------
        #  Train Generators
        # -----------------
        optimizer_G.zero_grad()

        # Generate fake images
        fake_B = netG_A2B(real_A)
        fake_A = netG_B2A(real_B)

        # Identity loss
        same_B = netG_A2B(real_B)
        loss_identity_B = criterion_identity(same_B, real_B) * 5.0
        same_A = netG_B2A(real_A)
        loss_identity_A = criterion_identity(same_A, real_A) * 5.0

        # Adversarial loss
        pred_fake_B = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake_B, True)
        pred_fake_A = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake_A, True)

        # Cycle loss
        recovered_A = netG_B2A(fake_B)
        loss_cycle_ABA = criterion_cycle(recovered_A, real_A) * 10.0
        recovered_B = netG_A2B(fake_A)
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B) * 10.0

        # Total generator loss
        loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
        loss_G.backward()
        optimizer_G.step()

        # Print generator loss information every 10 batches

        # -----------------------
        #  Train Discriminator A
        # -----------------------
        optimizer_D_A.zero_grad()

        # Real loss
        pred_real_A = netD_A(real_A)
        loss_D_real_A = criterion_GAN(pred_real_A, True)

        # Fake loss
        pred_fake_A = netD_A(fake_A.detach())
        loss_D_fake_A = criterion_GAN(pred_fake_A, False)

        # Total loss
        loss_D_A = (loss_D_real_A + loss_D_fake_A) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        # -----------------------
        #  Train Discriminator B
        # -----------------------
        optimizer_D_B.zero_grad()

        # Real loss
        pred_real_B = netD_B(real_B)
        loss_D_real_B = criterion_GAN(pred_real_B, True)

        # Fake loss
        pred_fake_B = netD_B(fake_B.detach())
        loss_D_fake_B = criterion_GAN(pred_fake_B, False)

        # Total loss
        loss_D_B = (loss_D_real_B + loss_D_fake_B) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()
        torch.cuda.synchronize()

        losses_G.append(loss_G.item())
        losses_D_A.append(loss_D_A.item())
        losses_D_B.append(loss_D_B.item())
        
        # Log and plot progress at the end of each epoch
    print(f"End of Epoch {epoch+1}\n"
          f"Loss_G: {loss_G.item()}\n"
          f"Loss_D_A: {loss_D_A.item()}\n"
          f"Loss_D_B: {loss_D_B.item()}\n")

    # Plotting generator losses at the end of each epoch
    plt.figure(figsize=(10, 5))
    plt.plot(losses_G, label='Generator Loss')
    plt.title('Generator Training Losses Over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plotting discriminator losses at the end of each epoch
    plt.figure(figsize=(10, 5))
    plt.plot(losses_D_A, label='Discriminator A Loss')
    plt.plot(losses_D_B, label='Discriminator B Loss')
    plt.title('Discriminator Training Losses Over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


    fake_B_image_path = os.path.join(output_directory, f'fake_B_epoch_{epoch+1}.png')
    fake_A_image_path = os.path.join(output_directory, f'fake_A_epoch_{epoch+1}.png')
    
    # Save the generated images
    save_image(fake_B.data[:25], fake_B_image_path, nrow=5, normalize=True)
    save_image(fake_A.data[:25], fake_A_image_path, nrow=5, normalize=True)

    # Display the generated images using matplotlib
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    # Display fake_B
    fake_B_img = plt.imread(fake_B_image_path)
    ax[0].imshow(fake_B_img)
    ax[0].set_title('Generated Image B')
    ax[0].axis('off')
    
    # Display fake_A
    fake_A_img = plt.imread(fake_A_image_path)
    ax[1].imshow(fake_A_img)
    ax[1].set_title('Generated Image A')
    ax[1].axis('off')
    
    plt.show()
    # Optionally save the model for future use
    torch.save(netG_A2B.module.state_dict(), f'/workspace/weights/Generator1/Generator1_game2movie{epoch}.pth')
    torch.save(netG_B2A.module.state_dict(), '/workspace/weights/Generator1_movie2game.pth')
    torch.save(netD_A.module.state_dict(), '/workspace/weights/Discriminator1Movie.pth')
    torch.save(netD_B.module.state_dict(), '/workspace/weights/Discriminator1Game.pth')


CycleGAN training loop below is for 2.2 as it uses frames mentioned in 1.3

In [ ]:
# Load datasets
dataset_A = datasets.ImageFolder('/workspace/training/gameTrain', transform=transform)
dataset_B = datasets.ImageFolder('/workspace/training/movieTrain', transform=transform)

total_batch_size = 24

loader_A = DataLoader(dataset_A, batch_size=total_batch_size, shuffle=True, num_workers=12)
loader_B = DataLoader(dataset_B, batch_size=total_batch_size, shuffle=True, num_workers=12)


# Ensure the output directory exists
output_directory = '/workspace/outputs2.2'
os.makedirs(output_directory, exist_ok=True)

losses_G, losses_D_A, losses_D_B = [], [], []

epochs = 100  # Number of epochs to train
for epoch in range(epochs):
    for i, (real_A, real_B) in enumerate(zip(loader_A, loader_B)):
        real_A = real_A[0].to(device)  # Move data to device
        real_B = real_B[0].to(device)

        # -----------------
        #  Train Generators
        # -----------------
        optimizer_G.zero_grad()

        # Generate fake images
        fake_B = netG_A2B(real_A)
        fake_A = netG_B2A(real_B)

        # Identity loss
        same_B = netG_A2B(real_B)
        loss_identity_B = criterion_identity(same_B, real_B) * 5.0
        same_A = netG_B2A(real_A)
        loss_identity_A = criterion_identity(same_A, real_A) * 5.0

        # Adversarial loss
        pred_fake_B = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake_B, True)
        pred_fake_A = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake_A, True)

        # Cycle loss
        recovered_A = netG_B2A(fake_B)
        loss_cycle_ABA = criterion_cycle(recovered_A, real_A) * 10.0
        recovered_B = netG_A2B(fake_A)
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B) * 10.0

        # Total generator loss
        loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
        loss_G.backward()
        optimizer_G.step()

        # Print generator loss information every 10 batches

        # -----------------------
        #  Train Discriminator A
        # -----------------------
        optimizer_D_A.zero_grad()

        # Real loss
        pred_real_A = netD_A(real_A)
        loss_D_real_A = criterion_GAN(pred_real_A, True)

        # Fake loss
        pred_fake_A = netD_A(fake_A.detach())
        loss_D_fake_A = criterion_GAN(pred_fake_A, False)

        # Total loss
        loss_D_A = (loss_D_real_A + loss_D_fake_A) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        # -----------------------
        #  Train Discriminator B
        # -----------------------
        optimizer_D_B.zero_grad()

        # Real loss
        pred_real_B = netD_B(real_B)
        loss_D_real_B = criterion_GAN(pred_real_B, True)

        # Fake loss
        pred_fake_B = netD_B(fake_B.detach())
        loss_D_fake_B = criterion_GAN(pred_fake_B, False)

        # Total loss
        loss_D_B = (loss_D_real_B + loss_D_fake_B) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()
        torch.cuda.synchronize()

        losses_G.append(loss_G.item())
        losses_D_A.append(loss_D_A.item())
        losses_D_B.append(loss_D_B.item())
        
        # Log and plot progress at the end of each epoch
    print(f"End of Epoch {epoch+1}\n"
          f"Loss_G: {loss_G.item()}\n"
          f"Loss_D_A: {loss_D_A.item()}\n"
          f"Loss_D_B: {loss_D_B.item()}\n")

    # Plotting generator losses at the end of each epoch
    plt.figure(figsize=(10, 5))
    plt.plot(losses_G, label='Generator Loss')
    plt.title('Generator Training Losses Over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plotting discriminator losses at the end of each epoch
    plt.figure(figsize=(10, 5))
    plt.plot(losses_D_A, label='Discriminator A Loss')
    plt.plot(losses_D_B, label='Discriminator B Loss')
    plt.title('Discriminator Training Losses Over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


    fake_B_image_path = os.path.join(output_directory, f'fake_B_epoch_{epoch+1}.png')
    fake_A_image_path = os.path.join(output_directory, f'fake_A_epoch_{epoch+1}.png')
    
    # Save the generated images
    save_image(fake_B.data[:25], fake_B_image_path, nrow=5, normalize=True)
    save_image(fake_A.data[:25], fake_A_image_path, nrow=5, normalize=True)

    # Display the generated images using matplotlib
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    # Display fake_B
    fake_B_img = plt.imread(fake_B_image_path)
    ax[0].imshow(fake_B_img)
    ax[0].set_title('Generated Image B')
    ax[0].axis('off')
    
    # Display fake_A
    fake_A_img = plt.imread(fake_A_image_path)
    ax[1].imshow(fake_A_img)
    ax[1].set_title('Generated Image A')
    ax[1].axis('off')
    
    plt.show()
    # Optionally save the model for future use
    torch.save(netG_A2B.module.state_dict(), f'/workspace/weights/Generator2/Generator2_game2movie{epoch}.pth')
    torch.save(netG_B2A.module.state_dict(), '/workspace/weights/Generator2_movie2game.pth')
    torch.save(netD_A.module.state_dict(), '/workspace/weights/Discriminator2Movie.pth')
    torch.save(netD_B.module.state_dict(), '/workspace/weights/Discriminator2Game.pth')

Video Processing

In [3]:
import cv2
import os
import torch
from torchvision import transforms
from PIL import Image

class Generator(nn.Module):
    """generator with downsampling, residual blocks, and upsampling components."""
    def __init__(self, input_channels, output_channels, n_residual_blocks=9):
        super(Generator, self).__init__()

        # Initial convolutional block
        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_channels, 64, kernel_size=7),
                 nn.InstanceNorm2d(64),
                 nn.ReLU(inplace=True)]

        # Downsampling
        in_features = 64
        out_features = in_features * 2
        for _ in range(2):
            model += [nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1),
                      nn.InstanceNorm2d(out_features),
                      nn.ReLU(inplace=True)]
            in_features = out_features
            out_features = in_features * 2

        # Residual blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features // 2
        for _ in range(2):
            model += [nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1),
                      nn.InstanceNorm2d(out_features),
                      nn.ReLU(inplace=True)]
            in_features = out_features
            out_features = in_features // 2

        # Output layer
        model += [nn.ReflectionPad2d(3),
                  nn.Conv2d(64, output_channels, kernel_size=7),
                  nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

def load_model(filepath, input_channels, output_channels, device):
    print(f"Loading model from {filepath}")
    model = Generator(input_channels, output_channels)
    model.load_state_dict(torch.load(filepath)) 
    model.to(device)
    model.eval()
    return model

def process_video(input_video_path, output_dir, model, device):
    print(f"Opening video file {input_video_path}")
    cap = cv2.VideoCapture(input_video_path)
    success, frame = cap.read()
    count = 0

    if not success:
        print("Failed to read video")
        return

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    while success:
        frame_path = os.path.join(output_dir, f'frame{count:04d}.jpg')
        cv2.imwrite(frame_path, frame)
        print(f"Saved frame {count} at {frame_path}")

        img = Image.open(frame_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)  # Move to GPU
        with torch.no_grad():
            output = model(img_tensor).squeeze(0).cpu()  # Process on GPU and move back to CPU
        output_img = transforms.ToPILImage()(output).convert("RGB")
        output_img.save(frame_path)
        print(f"Processed and saved frame {count}")

        success, frame = cap.read()
        count += 1

    cap.release()
    print("Finished processing all frames.")

def compile_video(frames_dir, output_video_path, fps):
    print(f"Compiling video from frames in {frames_dir}")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.jpg')])
    frame = cv2.imread(files[0])
    height, width, layers = frame.shape
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for file in files:
        video.write(cv2.imread(file))
        print(f"Adding {file} to video.")

    video.release()
    print(f"Video compiled and saved as {output_video_path}")

In [18]:
if __name__ == '__main__':
    video_path = '/workspace/Test.mp4'
    model_path = '/workspace/weights/Generator1/Generator1_game2movie99.pth'
    output_dir = '/workspace/processed_frames'
    result_video = 'result1.mp4'
    fps = 30
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    model = load_model(model_path, 3, 3, device)
    process_video(video_path, output_dir, model, device)
    compile_video(output_dir, result_video, fps)

Loading model from /workspace/weights/Generator1/Generator1_game2movie99.pth
Opening video file /workspace/Test.mp4
Saved frame 0 at /workspace/processed_frames/frame0000.jpg
Processed and saved frame 0
Saved frame 1 at /workspace/processed_frames/frame0001.jpg
Processed and saved frame 1
Saved frame 2 at /workspace/processed_frames/frame0002.jpg
Processed and saved frame 2
Saved frame 3 at /workspace/processed_frames/frame0003.jpg
Processed and saved frame 3
Saved frame 4 at /workspace/processed_frames/frame0004.jpg
Processed and saved frame 4
Saved frame 5 at /workspace/processed_frames/frame0005.jpg
Processed and saved frame 5
Saved frame 6 at /workspace/processed_frames/frame0006.jpg
Processed and saved frame 6
Saved frame 7 at /workspace/processed_frames/frame0007.jpg
Processed and saved frame 7
Saved frame 8 at /workspace/processed_frames/frame0008.jpg
Processed and saved frame 8
Saved frame 9 at /workspace/processed_frames/frame0009.jpg
Processed and saved frame 9
Saved frame 10

In [6]:
if __name__ == '__main__':
    video_path = '/workspace/Test.mp4'
    model_path = '/workspace/weights/Generator2/Generator2_game2movie98.pth'
    output_dir = '/workspace/processed_frames'
    result_video = 'result2.mp4'
    fps = 30
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    model = load_model(model_path, 3, 3, device)
    process_video(video_path, output_dir, model, device)
    compile_video(output_dir, result_video, fps)

Loading model from /workspace/weights/Generator2/Generator2_game2movie98.pth
Opening video file /workspace/Test.mp4
Saved frame 0 at /workspace/processed_frames/frame0000.jpg
Processed and saved frame 0
Saved frame 1 at /workspace/processed_frames/frame0001.jpg
Processed and saved frame 1
Saved frame 2 at /workspace/processed_frames/frame0002.jpg
Processed and saved frame 2
Saved frame 3 at /workspace/processed_frames/frame0003.jpg
Processed and saved frame 3
Saved frame 4 at /workspace/processed_frames/frame0004.jpg
Processed and saved frame 4
Saved frame 5 at /workspace/processed_frames/frame0005.jpg
Processed and saved frame 5
Saved frame 6 at /workspace/processed_frames/frame0006.jpg
Processed and saved frame 6
Saved frame 7 at /workspace/processed_frames/frame0007.jpg
Processed and saved frame 7
Saved frame 8 at /workspace/processed_frames/frame0008.jpg
Processed and saved frame 8
Saved frame 9 at /workspace/processed_frames/frame0009.jpg
Processed and saved frame 9
Saved frame 10